In [91]:
from langchain.document_loaders import CSVLoader
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from dotenv import load_dotenv
load_dotenv()
from langchain.callbacks import StdOutCallbackHandler



import os
llm=OpenAI(temperature=0.2,api_key=os.environ["OPENAI_API_KEY"])


vectordb_file_path = "vectordb"
instructor_embeddings = OpenAIEmbeddings(api_key=os.environ["OPENAI_API_KEY"])
def load_data():
    loader = CSVLoader(file_path="data_2.csv",source_column="Sentence")
    documents = loader.load()
    # Create a FAISS instance for vector database from 'data'
    vectordb = FAISS.from_documents(documents=documents,
                                    embedding=instructor_embeddings)

    # Save vector database locally
    vectordb.save_local(vectordb_file_path)

def get_response():
    vectordb = FAISS.load_local(vectordb_file_path, instructor_embeddings)
    retriever = vectordb.as_retriever(score_threshold=0.8,k=3)

    prompt_template = """Given the following context and a question, generate an answer based on this context only.
    In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
    If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

    CONTEXT: {context}

    QUESTION: {question}"""

    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )


    response = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", 
                                         input_key="query",
                                         retriever=retriever, return_source_documents=True,
                                         chain_type_kwargs={"prompt": PROMPT},
                                         callbacks=[StdOutCallbackHandler()],
                                         verbose=True,
                                         )

    return response



In [39]:
load_data()

In [92]:
chain=get_response()

In [93]:
out_dict=chain("do u have any offers in jewelry category in Cartier brand within 20 days")



> Entering new RetrievalQA chain...

> Finished chain.


In [94]:
result=out_dict["result"]

In [95]:
suggestions=out_dict["source_documents"]

In [96]:
out_dict

{'query': 'do u have any offers in jewelry category in Cartier brand within 20 days',
 'result': '?\n\nYes, we have an offer in the jewelry category from Cartier brand that is expiring in 20 days. Get 20 percent off on Cartier jewelry! Hurry, limited offer - only 20 days left!',
 'source_documents': [Document(page_content='category: jewelry\nbrand: Cartier\noffers: 20 percent\nexpiring_in_days: 20\nSentence: ðŸŒŸ Grab 20 percent off on Cartier jewelry! Limited offer, only 20 days left!', metadata={'source': 'ðŸŒŸ Grab 20 percent off on Cartier jewelry! Limited offer, only 20 days left!', 'row': 1153}),
  Document(page_content='category: jewelry\nbrand: Cartier\noffers: 20 percent\nexpiring_in_days: 28\nSentence: ðŸŒŸ Grab 20 percent off on Cartier jewelry! Limited offer, only 28 days left!', metadata={'source': 'ðŸŒŸ Grab 20 percent off on Cartier jewelry! Limited offer, only 28 days left!', 'row': 1341}),
  Document(page_content='category: jewelry\nbrand: Cartier\noffers: 20 percent\n

In [97]:
suggestions[0].metadata["row"]

1153

In [98]:
def getid(t):
    return t.metadata["row"]

In [103]:
import pandas as pd
df=pd.read_csv("data_2.csv")

ids=[]
for i in suggestions:
    ids.append(df.iloc[getid(i),-1])
    print("\n")

In [104]:
suggestion_df=pd.DataFrame(ids)

In [105]:
result

'?\n\nYes, we have an offer in the jewelry category from Cartier brand that is expiring in 20 days. Get 20 percent off on Cartier jewelry! Hurry, limited offer - only 20 days left!'

In [106]:
suggestion_df.head()

,0
0,🌟 Grab 20 percent off on Cartier jewelry! Limi...
1,🌟 Grab 20 percent off on Cartier jewelry! Limi...
2,🌟 Grab 20 percent off on Cartier jewelry! Limi...
3,🌟 Grab 20 percent off on Cartier jewelry! Limi...
